In [ ]:
import random
import re
from math import pi
import os
import shutil

In [1]:
def find_tab (line):#uesd to find tab in the front of each line and reture the tabs to ensure the grammar correct
    i = 0
    tab = ""
    while line[i] == " ":
        tab = tab + line[i]
        i = i + 1
    return tab

def execution(pyfile_name):# execute quantum circuits in the formate of .py and catch possible errors.
    try:
        os.system('python ' + pyfile_name)
    except Exception as e:
        print("OS error:" + str(e))

In [2]:
def simulator_to_state_vector (file_in):# change the backend of seed program to state_vector

    change_file_name=re.compile("startPyquil[0-9]+")
    readfile = open(file_in)
    file_out_name= "startPyquil_statevector_"+ re.sub("\D","",file_in)+".py"
    bii=re.compile("bitstrings =")
    run=re.compile("run_and_measure")
    qvm_pattern=re.compile("4q-qvm")
    pri=re.compile("print")

    writefile = open(file_out_name,"w")
    line = readfile.readline()
    while line:
        if qvm_pattern.search(line):
            tab=find_tab(line)
            writefile.write(tab+"state = conn.wavefunction(prog)\n")
            line = readfile.readline()
            continue
            
        if run.search(line):
            line = readfile.readline()
            continue
            
        if bii.search(line):
            line = readfile.readline()
            continue
            
        if pri.search(line):
            tab=find_tab(line)
            writefile.write(tab+"print(state.get_outcome_probs(),file=writefile)\n")
            line = readfile.readline()
            continue
            
        if change_file_name.search(line):
            number=re.sub("\D","",re.findall(change_file_name,line)[0] )
            cc1=re.sub(change_file_name,"startPyquil_statevector_"+str(number),line)
            writefile.write(cc1)
        else:
            writefile.write(line)
        line = readfile.readline()
    readfile.close()
    writefile.close()
    return file_out_name
    
def simulator_to_prag (file_in):# change the backend of seed program to pragma
    pragma_pattern = ['NAIVE','PARTIAL','GREEDY']
    program_begin = re.compile("Program\(\)")
    change_file_name=re.compile("startPyquil[0-9]+")
    readfile = open(file_in)
    file_out_name= "startPyquil_pragma_"+ re.sub("\D","",file_in)+".py"
    writefile = open(file_out_name,"w")
    line = readfile.readline()
    while line:
        
        if program_begin.search(line):
            tab=find_tab(line)
            cc="prog = Program('PRAGMA INITIAL_REWIRING \""
            writefile.write(tab+cc+random.choice(pragma_pattern)+"\"') # circuit begin")
            line = readfile.readline()
            continue
            
        if change_file_name.search(line):
            number=re.sub("\D","",re.findall(change_file_name,line)[0] )
            cc1=re.sub(change_file_name,"startPyquil_pragma_"+str(number),line)
            writefile.write(cc1)
        else:
            writefile.write(line)
        line = readfile.readline()
    readfile.close()
    writefile.close()
    return file_out_name

In [3]:

number_sw= '\([0-9]+,[0-9]+\)'
number= '\([0-9]+\)'
number_1=" # number=[0-9]+"

###the following functions is uses to replace one gate to the euqivalent gates 
def cx2h(tab,line,total_number):
    sentence= tab+"prog += H("+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[1]+') # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog += CZ("+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[0]+ ','+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[1]+') # number='+ str(total_number+1) +"\n"+\
                tab+"prog += H("+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[1]+') # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2    

def cz2h(tab,line,total_number):
    sentence= tab+"prog += H("+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[1]+') # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog += CNOT("+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[0]+ ','+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[1]+') # number='+ str(total_number+1) +"\n"+\
                tab+"prog += H("+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[1]+') # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2    

def swap2cx(tab,line,total_number):
    sentence= tab+"prog += CNOT("+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[0]+ ','+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[1]+') # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog += CNOT("+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[0]+ ','+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[1]+') # number='+ str(total_number+1) +"\n"+\
                tab+"prog += CNOT("+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[0]+ ','+ re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[1]+') # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2    
def x2cx(tab,line,total_number):
    currentnumber= re.sub("\D","",re.findall(number,line)[0])
    if int(currentnumber)==0:
        added_number=1
    else:
        added_number=0

    sentence= tab+"prog += CNOT("+ str(added_number) + ','+ str(currentnumber)+') # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
    tab+"prog += X("+  str(currentnumber)+ ') # number='+ str(total_number+1) +"\n"+\
    tab+"prog += CNOT("+ str(added_number) + ','+ str(currentnumber)+') # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2

def z2cx(tab,line,total_number):
    
    currentnumber= re.sub("\D","",re.findall(number,line)[0])
    if int(currentnumber)==0:
        added_number=1
    else:
        added_number=0
        
    sentence= tab+"prog += CNOT("+ str(currentnumber) + ','+ str(added_number)+') # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
    tab+"prog += Z("+  str(currentnumber)+ ') # number='+ str(total_number+1) +"\n"+\
    tab+"prog += CNOT("+ str(currentnumber) + ','+ str(added_number)+') # number='+ str(total_number+2) +"\n"
    return sentence,total_number+2

def s2tt(tab,line,total_number):
    sentence= tab+"prog += T("+ re.sub("\D","",re.findall(number,line)[0])+ ') # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog += T("+ re.sub("\D","",re.findall(number,line)[0])+ ') # number='+ str(total_number+1) +"\n"
    return sentence,total_number+1

def z2ss(tab,line,total_number):
    sentence= tab+"prog += S("+ re.sub("\D","",re.findall(number,line)[0])+ ') # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog += S("+ re.sub("\D","",re.findall(number,line)[0])+ ') # number='+ str(total_number+1) +"\n"
    return sentence,total_number+1

def x2hssh(tab,line,total_number):
    sentence= tab+"prog += H("+ re.sub("\D","",re.findall(number,line)[0])+ ') # number='+ re.sub("\D","",re.findall(number_1,line)[0])+"\n"+\
            tab+"prog += S("+ re.sub("\D","",re.findall(number,line)[0])+ ') # number='+ str(total_number+1) +"\n"+\
    tab+"prog += S("+ re.sub("\D","",re.findall(number,line)[0])+ ') # number='+ str(total_number+2) +"\n"+\
    tab+"prog += H("+ re.sub("\D","",re.findall(number,line)[0])+ ') # number='+ str(total_number+3) +"\n"
    return sentence,total_number+3


In [4]:
gate_x = re.compile("prog \+= X")
gate_s = re.compile("prog \+= S\(")
gate_z = re.compile("prog \+= Z")
gate_swap = re.compile("prog \+= SWAP")
gate_cnot=re.compile("prog \+= CNOT")
gate_cz = re.compile("prog \+= CZ")

def makeeuqalfiles(filein,fileout):
    flag=0 #set the flage to detect wethere there is any changed gates                     
    readfile = open(filein)
    writefile = open("euqal_temp","w")
    total_number=0
    total_operation_id = re.compile("# total number=")
    line = readfile.readline()
    wirte_new_file_name=re.compile("open" )
    while line:
          #####  read each line in the file and randomly change the gates to euqivalent gates
        if wirte_new_file_name.search(line):
            writefile.write(re.sub("startPyquil[0-9]+",fileout.split('.')[0],line))
            line = readfile.readline()
            continue
            
        if total_operation_id.search(line):
            total_number=int(re.sub("\D","",line))
#             print(total_number)
            
        if gate_s.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
                writefile.write(s2tt(tab,line,total_number)[0])
                flag=1
                total_number=s2tt(tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue

        if gate_swap.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
                writefile.write(swap2cx(tab,line,total_number)[0])
                flag=1
                total_number=swap2cx(tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue
            
        if gate_x.search(line):
            if random.randint(0,5)>2:
                tab=find_tab(line)
                op=[x2cx,x2hssh]####randomly choose one combination of gates in the list of opearations
                idd=random.randint(0,1)
                writefile.write(op[idd](tab,line,total_number)[0])
                flag=1
                total_number=op[idd](tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue
            
        if gate_z.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
                op_1=[z2ss,z2cx]
                idd_1=random.randint(0,1)
                writefile.write(op_1[idd_1](tab,line,total_number)[0])
                flag=1
                total_number=op_1[idd_1](tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue 
            
            
        if gate_cnot.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
#                 op_2=[cx2h,cx2cnot]
#                 idd_2=random.randint(0,1)
                writefile.write(cx2h(tab,line,total_number)[0])
                flag=1
                total_number=cx2h(tab,line,total_number)[1]
            else:
                writefile.write(line)
            line = readfile.readline()
            continue
            
        if gate_cz.search(line):
            if random.randint(0,5)>3:
                tab=find_tab(line)
                writefile.write(cz2h(tab,line,total_number)[0])
                flag=1
                total_number=cz2h(tab,line,total_number)[1]
            else:
                writefile.write(line)
        else:
            writefile.write(line)
        line = readfile.readline()

    readfile.close()
    writefile.close()
    
    readfile_11 = open('euqal_temp')#######in order to make the totoal number correct ,should read the file again to change the totoal number
    writefile_11 = open(fileout,"w")
    line_11 = readfile_11.readline()
    while line_11:
        if total_operation_id.search(line_11):
            writefile_11.write("# total number="+str(total_number)+"\n")
        else:
            writefile_11.write(line_11)
        line_11 = readfile_11.readline()
    readfile_11.close()
    writefile_11.close()
    os.remove("./euqal_temp")
    
    
    if flag==0:
        return 0
    else:
        return total_number  

In [5]:
def generate_add(tab,qubit_number,total_number):#####create a sentence which uses for adding method 
    gate_set_pyquil = ["prog += H(", "prog += X(","prog += Y(", "prog += Z(","prog += RX(", "prog += CNOT(", "prog += SWAP("]    
    q_number=random.randint(0,qubit_number-1)
    gatename=random.choice(gate_set_pyquil)
#     gatename="prog.cx"
    if gatename=="prog += H(" or gatename=="prog += X(" or gatename=="prog += Y(" or gatename=="prog += Z(":
        sentence=tab+gatename+ str(q_number) +")"+ " # number="+ str(total_number+1)
    elif gatename=="prog += RX(":####for rx gate, it should add a random angle.
        sentence=tab+gatename + str((random.randint(-1000,1000))/1000 *pi) +","  + str(q_number) +")"+ " # number="+ str(total_number+1) 
    elif gatename== "prog += CNOT(" or gatename== "prog += SWAP(":
        q1=random.randint(1,qubit_number-1)
        q2=random.randint(1,qubit_number-1)
        while q1==q2:
            q2=random.randint(1,qubit_number-1)
        if qubit_number==2:
            q1=0
            q2=1
        sentence=tab+gatename+ str(q1)+  "," + str(q2)+ ") # number=" + str(total_number+1)
    return sentence

def generate_change(tab,qubit_number,total_number,line):###ues the sentence which is gengerated by "generate_add" to nutateh seed program and save the new file as a new .py file 
    single_gate_h = re.compile("prog \+= H")
    single_gate_x = re.compile("prog \+= X")
    single_gate_y = re.compile("prog \+= Y")
    single_gate_z = re.compile("prog \+= Z")
    double_gate_swap= re.compile("prog \+= SWAP")
    double_gate_cnot= re.compile("prog \+= CNOT")
    rotate_gate_rx = re.compile("prog \+= RX")
    rotate_gate_ry = re.compile("prog \+= RY")
    rotate_gate_rz = re.compile("prog \+= RZ")
    number_sw= '\([0-9]+,[0-9]+\)'
    number= '\([0-9]+\)'
    number_1=" # number=[0-9]+"
    if single_gate_h.search(line):
        gate=random.choice(["prog += X(","prog += Y(","prog += Z("])
        sentence=tab+gate+ re.sub( "\)","" ,re.sub("\(","",re.findall(number,line)[0])) +")"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if single_gate_x.search(line):
        gate=random.choice(["prog += H(","prog += Y(","prog += Z("])
        sentence=tab+gate+ re.sub( "\)","" ,re.sub("\(","",re.findall(number,line)[0])) +")"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if single_gate_y.search(line):
        gate=random.choice(["prog += H(","prog += X(","prog += Z("])
        sentence=tab+gate+ re.sub( "\)","" ,re.sub("\(","",re.findall(number,line)[0])) +")"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if single_gate_z.search(line):
        gate=random.choice(["prog += H(","prog += Y(","prog += X("])
        sentence=tab+gate+re.sub( "\)","" ,re.sub("\(","",re.findall(number,line)[0])) +")"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    
    
    if double_gate_cnot.search(line):
        q1=random.randint(1,qubit_number-1)
        while q1==re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[0]:
            q1=random.randint(1,qubit_number-1)
        
        q2=random.randint(1,qubit_number-1)
        while q2==q1:
            q2=random.randint(1,qubit_number-1)
        sentence=tab+"prog += CNOT(" + str(q1)+  "," + str(q2)+ ") # number=" + re.sub("\D","",re.findall(number_1,line)[0])
    
    
    
    
    if double_gate_swap.search(line):
        q1=random.randint(1,qubit_number-1)
        while q1==re.sub( "\)","" ,re.sub("\(","",re.findall(number_sw,line)[0])).split(",")[0]:
            q1=random.randint(1,qubit_number-1)
        
        q2=random.randint(1,qubit_number-1)
        while q2==q1:
            q2=random.randint(1,qubit_number-1)
        sentence=tab+"prog += SWAP(" + str(q1)+  "," + str(q2)+ ") # number=" + re.sub("\D","",re.findall(number_1,line)[0])
    if rotate_gate_rx.search(line):
        q1=random.randint(1,qubit_number-1)
        gate=random.choice(["prog += RY(","prog += RZ("])
        sentence=tab+"prog."+gate+"("+str((random.randint(-1000,1000))/1000 *pi)+","+ str(q1) +")"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if rotate_gate_ry.search(line):
        q1=random.randint(1,qubit_number-1)
        gate=random.choice(["prog += RX(","prog += RX("])
        sentence=tab+"prog."+gate+"("+str((random.randint(-1000,1000))/1000 *pi)+","+ str(q1) +")"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    if rotate_gate_rz.search(line):
        q1=random.randint(1,qubit_number-1)
        gate=random.choice(["prog += RY(","prog += RX("])
        sentence=tab+gate+str((random.randint(-1000,1000))/1000 *pi)+","+ str(q1) +")"+ " # number="+ re.sub("\D","",re.findall(number_1,line)[0])
    return sentence
def exe_adding(add_sentence,add_place_number,totalnumber,inpyname,outpyname):###ues the sentence which is gengerated by "generate_add" to nutateh seed program and save the new file as a new .py file 
    readfile = open(inpyname)
    writefile = open(outpyname,"w")#set the name of new py file
    line = readfile.readline()
    total_number = re.compile("# total number=")
    add_place=re.compile("# number="+str(add_place_number) )
#     print(add_place)
    wirte_new_file_name=re.compile("open" )
    
    
    while line:

        if add_place.search(line):
            writefile.write(line+add_sentence+"\n")
            line = readfile.readline()
            continue
        if wirte_new_file_name.search(line):
            writefile.write(re.sub("startPyquil[0-9]+",outpyname.split('.')[0],line))
            line = readfile.readline()
            continue
            
        if total_number.search(line):
            writefile.write("# total number="+str(totalnumber+1)+"\n")
        else:
            writefile.write(line)
        line = readfile.readline()
        
    readfile.close()
    writefile.close()
    
def exe_changing(sentence,place_number,inpyname,outpyname):###ues the sentence which is gengerated by "generate_change" to nutateh seed program and save the new file as a new .py file 
    readfile = open(inpyname)
    writefile = open(outpyname,"w")
    line = readfile.readline()
    change_place=re.compile("# number="+str(place_number) )
    wirte_new_file_name=re.compile("open" )
    while line:
        if wirte_new_file_name.search(line):
            writefile.write(re.sub("startPyquil[0-9]+",outpyname.split('.')[0],line))
#             print(re.sub(wirte_new_file_name,outpyname.split('.')[0],line))
            line = readfile.readline()
            continue
            
        if change_place.search(line):
            writefile.write(sentence+"\n")
        else:
            writefile.write(line)
        line = readfile.readline()
        
    readfile.close()
    writefile.close()

def exe_deleting(totalnumber,place_number,inpyname,outpyname): ##find the position which shou be delete and delete the line, then change the totoal number and save as a new file
    readfile = open(inpyname)
    writefile = open("temp","w")
    line = readfile.readline()
    change_place=re.compile("# number="+str(place_number) )
    wirte_new_file_name=re.compile( "open" )
    total_number = re.compile("# total number=")
    
    while line:
        
        if total_number.search(line):
            writefile.write("# total number="+str(totalnumber-1)+"\n")
            line = readfile.readline()
            continue
        if wirte_new_file_name.search(line):
            writefile.write(re.sub("startPyquil[0-9]+",outpyname.split('.')[0],line))
            line = readfile.readline()
            continue
        if change_place.search(line):
            writefile.write("")
        else:
            writefile.write(line)
        line = readfile.readline()
        
    readfile.close()
    writefile.close()
    
    readfile_1 = open('temp')
    writefile_1 = open(outpyname,"w")
    number_patern=re.compile("# number=")
    i=1
    line_1 = readfile_1.readline()
    while line_1:
        if number_patern.search(line_1):
            writefile_1.write(re.sub( "# number=[0-9]+","# number="+str(i),line_1))
            i +=1 
        else:
            writefile_1.write(line_1)
        line_1 = readfile_1.readline()
    readfile_1.close()
    writefile_1.close()
    os.remove("./temp")
    
def mute(file_in,file_out): ###### this function includes the formal functions to mutate the seed program to a new seed program
    qubit_number_patter = re.compile("# qubit number=")
    operation_id = re.compile("# number=")
    nu = re.compile("# number=[0-9]+")
    total_operation_id = re.compile("# total number=")
    qubit_number = 0
    total_number = 0
    mutation_number = 1
    flag = 0
    gate_set_pyquil = ["prog += H(", "prog += X(","prog += Y(", "prog += Z(","prog += RX(", "prog += CNOT(", "prog += SWAP("]    
    readfile = open(file_in)
    line = readfile.readline()
    
    while line:
        if qubit_number_patter.search(line):
            qubit_number= int(re.sub("\D","",line))
        if total_operation_id.search(line):
            total_number=int(re.sub("\D","",line))
            if total_number==1:
                mutation_number = 1
            else:
                mutation_number=random.randint(1,total_number-1)#mutation_number randomly decides which line to use for mutation 
        if operation_id.search(line):
#             print(re.findall(operation_id,line))
            flag=int( re.sub("\D","",re.findall(nu,line)[0])) ##uesd the find the line which is decided by mutation_number
            
        if flag == mutation_number: # read each line of the file , and adopt different operation for each line
            if find_tab(line) != "":
                tab=find_tab(line)
            if total_number != 1: ####randomly chooes add, delete or change operation
                muta_choose=random.randint(1,3)
            else:
                muta_choose=random.randint(1,2)
#             muta_choose=2
            if muta_choose == 1:
                print("adding")
                resu=generate_add(tab,qubit_number,total_number)
                exe_adding(resu,mutation_number,total_number,file_in,file_out)
                readfile.close()
                return 1
            elif muta_choose == 2:
                print("changing")
#                 print(mutation_number)
                resu=generate_change(tab,qubit_number,total_number,line)
#                 print(resu)
#                 print(file_out)
                exe_changing(resu,mutation_number,file_in,file_out)
                
                readfile.close()
                return 1
            else:
                print("delecting")
                exe_deleting(total_number,mutation_number,file_in,file_out)
                readfile.close()
                return 1
        line = readfile.readline()
    
    readfile.close()
    return 0

In [6]:
def child_loop():  #####this loop is uesd to generate euqivalent circuits and change the backend of them
    flag=210
    pipei=re.compile("startPyquil[0-9]+")
    while flag<232: ####### this number minus the initial number of flag is the number of equivalent circuits of each seed program
        wait=[]
        for file in os.listdir("."):
            if pipei.search(file):
                wait.append(file)
        choose_file=random.choice(wait)
        times=0
        while times<10:
            bb=makeeuqalfiles(choose_file,"startPyquil"+str(flag)+".py")
            if bb==0:
                bb=makeeuqalfiles(choose_file,"startPyquil"+str(flag)+".py")
                times +=1
            else:
                break
        name=simulator_to_state_vector("startPyquil"+str(flag)+".py")
        simulator_to_prag("startPyquil"+str(flag)+".py")
        print("generate from"+str(choose_file)+"  to  "+str(flag))
        flag += 1

####different from Qiskit and Cirq, this qss has not method to judge the repeat circuit


In [7]:
%%time
whole_loop=1

py_file=re.compile("start")
####this is the main function of system.

while whole_loop<100: ###ues this number to control the numbers of mutation
    seed_choose=  random.choice(os.listdir("seed") )
    print("generate seed file from"+str(seed_choose)+"  to  "+str(whole_loop))
    mute( "seed/"+seed_choose,"startPyquil"+str(whole_loop)+".py")
    simulator_to_state_vector("startPyquil"+str(whole_loop)+".py")
    simulator_to_prag( "startPyquil"+str(whole_loop)+".py" )
    shutil.copy("startPyquil"+str(whole_loop)+".py","./seed")
    print("---------------------------------------------------")
    child_loop()    
    if os.path.exists(str(whole_loop)) is True:######after each iteration, move all the generated files to a new folder  for further calculation
        shutil.rmtree(str(whole_loop))
    os.mkdir(str(whole_loop) )
    files=os.listdir(".")
    for file in files:
        if py_file.search(file) :
            shutil.move(str(file),str(whole_loop))
    whole_loop += 1


generate seed file fromstartPyquil0.py  to  1
adding
---------------------------------------------------
generate fromstartPyquil1.py  to  210
generate fromstartPyquil210.py  to  211
generate fromstartPyquil210.py  to  212
generate fromstartPyquil212.py  to  213
generate fromstartPyquil211.py  to  214
generate fromstartPyquil211.py  to  215
generate fromstartPyquil211.py  to  216
generate fromstartPyquil1.py  to  217
generate fromstartPyquil215.py  to  218
generate fromstartPyquil216.py  to  219
generate fromstartPyquil210.py  to  220
generate fromstartPyquil219.py  to  221
generate fromstartPyquil1.py  to  222
generate fromstartPyquil1.py  to  223
generate fromstartPyquil221.py  to  224
generate fromstartPyquil217.py  to  225
generate fromstartPyquil213.py  to  226
generate fromstartPyquil225.py  to  227
generate fromstartPyquil219.py  to  228
generate fromstartPyquil219.py  to  229
generate fromstartPyquil226.py  to  230
generate fromstartPyquil229.py  to  231
generate seed file from

generate fromstartPyquil216.py  to  217
generate fromstartPyquil211.py  to  218
generate fromstartPyquil9.py  to  219
generate fromstartPyquil9.py  to  220
generate fromstartPyquil220.py  to  221
generate fromstartPyquil210.py  to  222
generate fromstartPyquil211.py  to  223
generate fromstartPyquil210.py  to  224
generate fromstartPyquil220.py  to  225
generate fromstartPyquil214.py  to  226
generate fromstartPyquil219.py  to  227
generate fromstartPyquil212.py  to  228
generate fromstartPyquil210.py  to  229
generate fromstartPyquil9.py  to  230
generate fromstartPyquil220.py  to  231
generate seed file fromstartPyquil6.py  to  10
adding
---------------------------------------------------
generate fromstartPyquil10.py  to  210
generate fromstartPyquil210.py  to  211
generate fromstartPyquil210.py  to  212
generate fromstartPyquil212.py  to  213
generate fromstartPyquil10.py  to  214
generate fromstartPyquil10.py  to  215
generate fromstartPyquil210.py  to  216
generate fromstartPyqui

generate fromstartPyquil221.py  to  224
generate fromstartPyquil221.py  to  225
generate fromstartPyquil219.py  to  226
generate fromstartPyquil213.py  to  227
generate fromstartPyquil18.py  to  228
generate fromstartPyquil227.py  to  229
generate fromstartPyquil223.py  to  230
generate fromstartPyquil212.py  to  231
generate seed file fromstartPyquil4.py  to  19
adding
---------------------------------------------------
generate fromstartPyquil19.py  to  210
generate fromstartPyquil19.py  to  211
generate fromstartPyquil19.py  to  212
generate fromstartPyquil19.py  to  213
generate fromstartPyquil19.py  to  214
generate fromstartPyquil212.py  to  215
generate fromstartPyquil211.py  to  216
generate fromstartPyquil213.py  to  217
generate fromstartPyquil211.py  to  218
generate fromstartPyquil218.py  to  219
generate fromstartPyquil218.py  to  220
generate fromstartPyquil212.py  to  221
generate fromstartPyquil218.py  to  222
generate fromstartPyquil212.py  to  223
generate fromstartPy

generate fromstartPyquil212.py  to  225
generate fromstartPyquil220.py  to  226
generate fromstartPyquil215.py  to  227
generate fromstartPyquil223.py  to  228
generate fromstartPyquil213.py  to  229
generate fromstartPyquil27.py  to  230
generate fromstartPyquil27.py  to  231
generate seed file fromstartPyquil5.py  to  28
changing
---------------------------------------------------
generate fromstartPyquil28.py  to  210
generate fromstartPyquil28.py  to  211
generate fromstartPyquil210.py  to  212
generate fromstartPyquil28.py  to  213
generate fromstartPyquil212.py  to  214
generate fromstartPyquil214.py  to  215
generate fromstartPyquil214.py  to  216
generate fromstartPyquil212.py  to  217
generate fromstartPyquil216.py  to  218
generate fromstartPyquil210.py  to  219
generate fromstartPyquil213.py  to  220
generate fromstartPyquil210.py  to  221
generate fromstartPyquil211.py  to  222
generate fromstartPyquil216.py  to  223
generate fromstartPyquil222.py  to  224
generate fromstar

generate fromstartPyquil210.py  to  211
generate fromstartPyquil211.py  to  212
generate fromstartPyquil212.py  to  213
generate fromstartPyquil210.py  to  214
generate fromstartPyquil211.py  to  215
generate fromstartPyquil210.py  to  216
generate fromstartPyquil215.py  to  217
generate fromstartPyquil216.py  to  218
generate fromstartPyquil211.py  to  219
generate fromstartPyquil212.py  to  220
generate fromstartPyquil214.py  to  221
generate fromstartPyquil212.py  to  222
generate fromstartPyquil212.py  to  223
generate fromstartPyquil212.py  to  224
generate fromstartPyquil220.py  to  225
generate fromstartPyquil225.py  to  226
generate fromstartPyquil221.py  to  227
generate fromstartPyquil212.py  to  228
generate fromstartPyquil224.py  to  229
generate fromstartPyquil212.py  to  230
generate fromstartPyquil218.py  to  231
generate seed file fromstartPyquil33.py  to  37
changing
---------------------------------------------------
generate fromstartPyquil37.py  to  210
generate fro

...

generate fromstartPyquil210.py  to  212
generate fromstartPyquil63.py  to  213
generate fromstartPyquil210.py  to  214
generate fromstartPyquil214.py  to  215
generate fromstartPyquil63.py  to  216
generate fromstartPyquil215.py  to  217
generate fromstartPyquil63.py  to  218
generate fromstartPyquil213.py  to  219
generate fromstartPyquil213.py  to  220
generate fromstartPyquil218.py  to  221
generate fromstartPyquil219.py  to  222
generate fromstartPyquil216.py  to  223
generate fromstartPyquil221.py  to  224
generate fromstartPyquil210.py  to  225
generate fromstartPyquil223.py  to  226
generate fromstartPyquil220.py  to  227
generate fromstartPyquil222.py  to  228
generate fromstartPyquil216.py  to  229
generate fromstartPyquil217.py  to  230
generate fromstartPyquil228.py  to  231
generate seed file fromstartPyquil17.py  to  64
adding
---------------------------------------------------
generate fromstartPyquil64.py  to  210
generate fromstartPyquil64.py  to  211
generate fromstart

generate fromstartPyquil226.py  to  228
generate fromstartPyquil211.py  to  229
generate fromstartPyquil228.py  to  230
generate fromstartPyquil210.py  to  231
generate seed file fromstartPyquil35.py  to  72
changing
---------------------------------------------------
generate fromstartPyquil72.py  to  210
generate fromstartPyquil72.py  to  211
generate fromstartPyquil210.py  to  212
generate fromstartPyquil72.py  to  213
generate fromstartPyquil210.py  to  214
generate fromstartPyquil211.py  to  215
generate fromstartPyquil211.py  to  216
generate fromstartPyquil212.py  to  217
generate fromstartPyquil211.py  to  218
generate fromstartPyquil211.py  to  219
generate fromstartPyquil72.py  to  220
generate fromstartPyquil213.py  to  221
generate fromstartPyquil210.py  to  222
generate fromstartPyquil215.py  to  223
generate fromstartPyquil213.py  to  224
generate fromstartPyquil210.py  to  225
generate fromstartPyquil214.py  to  226
generate fromstartPyquil210.py  to  227
generate fromst

generate fromstartPyquil219.py  to  228
generate fromstartPyquil222.py  to  229
generate fromstartPyquil219.py  to  230
generate fromstartPyquil215.py  to  231
generate seed file fromstartPyquil56.py  to  81
delecting
---------------------------------------------------
generate fromstartPyquil81.py  to  210
generate fromstartPyquil210.py  to  211
generate fromstartPyquil81.py  to  212
generate fromstartPyquil81.py  to  213
generate fromstartPyquil211.py  to  214
generate fromstartPyquil212.py  to  215
generate fromstartPyquil210.py  to  216
generate fromstartPyquil81.py  to  217
generate fromstartPyquil214.py  to  218
generate fromstartPyquil81.py  to  219
generate fromstartPyquil217.py  to  220
generate fromstartPyquil219.py  to  221
generate fromstartPyquil212.py  to  222
generate fromstartPyquil218.py  to  223
generate fromstartPyquil217.py  to  224
generate fromstartPyquil211.py  to  225
generate fromstartPyquil225.py  to  226
generate fromstartPyquil218.py  to  227
generate fromst

generate fromstartPyquil220.py  to  230
generate fromstartPyquil89.py  to  231
generate seed file fromstartPyquil14.py  to  90
adding
---------------------------------------------------
generate fromstartPyquil90.py  to  210
generate fromstartPyquil210.py  to  211
generate fromstartPyquil210.py  to  212
generate fromstartPyquil210.py  to  213
generate fromstartPyquil212.py  to  214
generate fromstartPyquil210.py  to  215
generate fromstartPyquil90.py  to  216
generate fromstartPyquil90.py  to  217
generate fromstartPyquil216.py  to  218
generate fromstartPyquil216.py  to  219
generate fromstartPyquil219.py  to  220
generate fromstartPyquil219.py  to  221
generate fromstartPyquil219.py  to  222
generate fromstartPyquil211.py  to  223
generate fromstartPyquil218.py  to  224
generate fromstartPyquil223.py  to  225
generate fromstartPyquil221.py  to  226
generate fromstartPyquil219.py  to  227
generate fromstartPyquil219.py  to  228
generate fromstartPyquil216.py  to  229
generate fromstar

generate fromstartPyquil219.py  to  227
generate fromstartPyquil220.py  to  228
generate fromstartPyquil216.py  to  229
generate fromstartPyquil213.py  to  230
generate fromstartPyquil213.py  to  231
generate seed file fromstartPyquil11.py  to  99
delecting
---------------------------------------------------
generate fromstartPyquil99.py  to  210
generate fromstartPyquil99.py  to  211
generate fromstartPyquil210.py  to  212
generate fromstartPyquil211.py  to  213
generate fromstartPyquil212.py  to  214
generate fromstartPyquil212.py  to  215
generate fromstartPyquil99.py  to  216
generate fromstartPyquil210.py  to  217
generate fromstartPyquil216.py  to  218
generate fromstartPyquil215.py  to  219
generate fromstartPyquil216.py  to  220
generate fromstartPyquil214.py  to  221
generate fromstartPyquil217.py  to  222
generate fromstartPyquil216.py  to  223
generate fromstartPyquil217.py  to  224
generate fromstartPyquil217.py  to  225
generate fromstartPyquil216.py  to  226
generate from